# 🚀 Universal OpenCode Server

**Works with Qwen, DeepSeek, and other models.**

This server translates model responses into proper tool calls that OpenCode can execute.

In [ ]:
#@title ⚙️ Configuration
#@markdown Choose your model:
MODEL = "qwen2.5-coder:7b"  #@param ["qwen2.5-coder:7b", "deepseek-coder-v2:16b", "codellama:13b", "mistral:7b"]
print(f"📦 Model: {MODEL}")

In [ ]:
#@title 📥 Install Dependencies
!nvidia-smi
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!pip install -q flask requests
print('\n✅ Dependencies installed')

In [ ]:
#@title 🤖 Start Ollama & Download Model
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print(f'Downloading {MODEL}...')
!ollama pull {MODEL}
print(f'\n✅ {MODEL} ready!')

In [ ]:
#@title 🔧 OpenCode Compatible API Server
from flask import Flask, request, jsonify
import requests as req
import json, time, uuid, threading, re

app = Flask(__name__)

# System prompt that teaches the model to output tool calls as JSON
SYSTEM_PROMPT = '''You are an expert coding assistant with access to tools.

## CRITICAL: Tool Usage Format
When you need to create files, run commands, or perform actions, you MUST output a tool call as JSON.

Available tools:
- file_write: Create or write a file. Args: path, content
- file_read: Read a file. Args: path
- shell: Run a command. Args: command
- list_files: List directory contents. Args: path (optional)
- search: Search for text in files. Args: pattern, path (optional)

## Tool Call Format
When using a tool, respond with ONLY this JSON block (no other text before or after):
```json
{"name": "tool_name", "arguments": {"arg1": "value1"}}
```

## Examples

User: "Create a hello world Python script"
```json
{"name": "file_write", "arguments": {"path": "hello.py", "content": "print('Hello, World!')"}}
```

User: "Run the script"
```json
{"name": "shell", "arguments": {"command": "python hello.py"}}
```

User: "What files are in this folder?"
```json
{"name": "list_files", "arguments": {}}
```

User: "Find all TODO comments"
```json
{"name": "search", "arguments": {"pattern": "TODO"}}
```

## When NOT to use tools
- For greetings ("Hi", "Hello") - respond with friendly text
- For questions about code - explain with text and code examples
- For help/documentation - provide helpful explanations

Be precise, helpful, and efficient.'''

# Custom tools that we support
CUSTOM_TOOLS = {'file_write', 'file_read', 'shell', 'list_files', 'search'}

def needs_tools(msg):
    """Check if the message needs tool execution."""
    m = msg.lower() if msg else ''
    actions = ['create', 'write', 'make', 'generate', 'new', 'save', 'add',
               'edit', 'modify', 'update', 'change', 'fix', 'refactor',
               'delete', 'remove', 'run', 'execute', 'test', 'build',
               'read', 'open', 'show', 'list', 'find', 'search', 'grep',
               'file', 'script', 'code', 'folder', 'directory']
    return any(a in m for a in actions)

def extract_tool(text):
    """Extract tool call from model response."""
    if not text:
        return None
    
    # Try JSON in code block
    m = re.search(r'```(?:json)?\s*([\s\S]*?)```', text)
    if m:
        try:
            d = json.loads(m.group(1).strip())
            if 'name' in d:
                return d['name'], json.dumps(d.get('arguments', {}))
        except: pass
    
    # Try raw JSON
    m = re.search(r'\{\s*"name"\s*:\s*"([^"]+)"\s*,\s*"arguments"\s*:\s*(\{[^{}]*\})', text)
    if m:
        return m.group(1), m.group(2)
    
    return None

@app.route('/v1/models', methods=['GET'])
def list_models():
    return jsonify({"object": "list", "data": [{"id": MODEL, "object": "model", "owned_by": "ollama"}]})

@app.route('/v1/chat/completions', methods=['POST'])
def chat():
    data = request.json
    messages = data.get('messages', [])
    
    # Get last user message
    user_msg = ''
    for m in reversed(messages):
        if m.get('role') == 'user' and m.get('content'):
            user_msg = str(m['content'])
            break
    
    use_tools = needs_tools(user_msg)
    print(f"[{time.strftime('%H:%M:%S')}] '{user_msg[:50]}' tools={use_tools}")
    
    # Build messages with system prompt
    msgs = [{'role': 'system', 'content': SYSTEM_PROMPT}]
    for m in messages:
        if m.get('role') != 'system':
            msgs.append(m)
    
    # Call Ollama
    try:
        r = req.post('http://localhost:11434/api/chat', json={
            'model': MODEL,
            'messages': msgs,
            'stream': False,
            'options': {'num_ctx': 8192}
        }, timeout=180)
        content = r.json().get('message', {}).get('content', '')
    except Exception as e:
        print(f"Error: {e}")
        content = "I'm ready to help! What would you like me to do?"
    
    # Try to extract tool call
    if use_tools and content:
        tool = extract_tool(content)
        if tool:
            name, args = tool
            print(f"  → Tool: {name}")
            return jsonify({
                "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
                "object": "chat.completion",
                "created": int(time.time()),
                "model": MODEL,
                "choices": [{
                    "index": 0,
                    "message": {
                        "role": "assistant",
                        "content": None,
                        "tool_calls": [{
                            "id": f"call_{uuid.uuid4().hex[:8]}",
                            "type": "function",
                            "function": {"name": name, "arguments": args}
                        }]
                    },
                    "finish_reason": "tool_calls"
                }],
                "usage": {"prompt_tokens": 100, "completion_tokens": 50, "total_tokens": 150}
            })
    
    # Clean and return text
    if content:
        # Remove any JSON blocks that we couldn't parse
        content = re.sub(r'```json[\s\S]*?```', '', content).strip()
        content = re.sub(r'\{\s*"name"[^}]+\}', '', content).strip()
    if not content:
        content = "Hello! I'm ready to help with your coding tasks."
    
    print(f"  → Text ({len(content)} chars)")
    return jsonify({
        "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
        "object": "chat.completion",
        "created": int(time.time()),
        "model": MODEL,
        "choices": [{"index": 0, "message": {"role": "assistant", "content": content}, "finish_reason": "stop"}],
        "usage": {"prompt_tokens": 100, "completion_tokens": len(content.split()), "total_tokens": 100 + len(content.split())}
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "model": MODEL})

# Start server
threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
time.sleep(2)
print(f'\n✅ API Server ready with {MODEL}')

In [ ]:
#@title 🧪 Test Tool Calling
import requests

print("Test 1: Create a file (should use tool)")
r = requests.post('http://localhost:5000/v1/chat/completions', json={
    'model': MODEL,
    'messages': [{'role': 'user', 'content': 'Create a Python file hello.py that prints Hello World'}]
}, timeout=120)
resp = r.json()['choices'][0]
if resp['message'].get('tool_calls'):
    tc = resp['message']['tool_calls'][0]['function']
    print(f"  ✅ Tool call: {tc['name']}")
    print(f"     Args: {tc['arguments'][:100]}...")
else:
    print(f"  ⚠️ Text response: {resp['message'].get('content', '')[:100]}...")

print("\nTest 2: Simple greeting (should return text)")
r = requests.post('http://localhost:5000/v1/chat/completions', json={
    'model': MODEL,
    'messages': [{'role': 'user', 'content': 'Hi there!'}]
}, timeout=120)
resp = r.json()['choices'][0]
if resp['message'].get('content'):
    print(f"  ✅ Text: {resp['message']['content'][:80]}...")
else:
    print(f"  ⚠️ Got tool call instead")

In [ ]:
#@title 🌐 Start Tunnel
import subprocess, re
from IPython.display import display, HTML

tunnel = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        m = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if m:
            url = m.group()
            display(HTML(f'''
            <div style="background:linear-gradient(135deg,#00b894,#00cec9);padding:30px;border-radius:20px">
                <h2 style="color:white;margin:0">🚀 OpenCode Server Ready!</h2>
                <p style="color:white;font-size:22px;font-family:monospace;margin:15px 0">{url}/v1</p>
                <hr style="border:none;border-top:1px solid rgba(255,255,255,0.3);margin:20px 0">
                <p style="color:white"><b>Model:</b> {MODEL}</p>
                <p style="color:white"><b>Custom Tools:</b> file_write, file_read, shell, list_files, search</p>
            </div>
            '''))
            break

for line in tunnel.stdout:
    print(line, end='')